# Feature Engineering I

## Stock Prediction

### Your name :     

### Kaggle Link for exercise :
* [Stock Price trend Prediction dataset](https://www.kaggle.com/aumashe/stock-ew/home) : Historical stock prices of Edwards Lifesciences Corporation (EW) 2000-03-26 ~ 2018-11-02

### Real Data for exercise :
* [내가 원하는 회사 데이터 다운 받을 곳!](https://finance.yahoo.com/quote/005930.KS/history?p=005930.KS&.tsrc=fin-srch-v1)
![how-to](https://github.com/RayleighKim/Example_datasets/blob/master/yahoo_finance.PNG?raw=true)

#### 실습목표<br>
* Feature Engineering을 통해 성능을 높여본다.

---------------
Rayleigh Kim @ D:plus


## Library Loading

In [0]:
'''
matplolib inline 명령어를 통해서
matplot으로 그리는 플롯들을 주피터 노트북 내에서 볼 수 있게 해준다.
포맷을 retina로 바꾸면 그래프의 화질이 훨씬 좋아진다.
'''
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

'''
라이브러리들을 불러오자.
'''
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt


from sklearn.metrics import mean_squared_error, r2_score

result_list = []

## Data Loading

In [0]:
data_path = 'https://raw.githubusercontent.com/RayleighKim/Example_datasets/master/EW.csv'
stock_raw = pd.read_csv(data_path)

In [0]:
# '''
# Stock_Edwards_Lifesciences_corportation.csv 를 업로드 하자.
# '''
# from google.colab import files
# files.upload

In [0]:
# stock_raw = pd.read_csv('Stock_Edwards_Lifesciences_corporation.csv')

In [0]:
print(stock_raw.shape)
stock_raw.head()

In [0]:
stock_raw[-100:].plot(x='Date', y='Close')

In [0]:
stock_raw[-20:].plot(x='Date', y='Close')
print('데이터의 최근 20일 간 그래프이다. 100개의 주식을 가지고 있다. \n 이상적이라면, 언제 팔고 언제 샀어야 했을까?')

## Data Pre-Processing, Feature Engineering

### Feature Engineering : 실습 포인트!

**판다스에서 배운 기능을 이용하여 새로운 변수들을 만들어 보자!**
* 삼성전자 예측을 위해, 다른 전자 회사의 정보를 가져와 **JOIN**할 수도 있다.
* **Rolling, Shift**기능은 수시로 쓰인다.
* NaN 이 있는 rows는 나중에 한꺼번에 처리할 예정.
  * 특별히 먼저 처리 해보고 싶은 사람은 해봐도 됨

* **Group by**는 장바구니 Feature Engineering 시간에!

In [0]:
stock = stock_raw.copy()

In [0]:
# ### ### Example ### ###

# ##### 처음에는 전체 주석을 해제하지 말고 그냥 실행하자.

# ##### 그 다음에는 주석을 해제해 보고 실행하자.


# # 하루 전 거래량을 오늘의 row에 붙인다.
# # 모델이 어제의 거래량도 반영했으면 좋겠다!
# stock['Volume_lag1'] = stock['Volume'].shift(periods=1)

# # 지난 1주일간 Low(최저가)의 minimum 값을 오늘의 row에 붙인다.
# # 오늘제외한 어제부터의 1주일간.
# # 아무래도 최저값이란 것이 가장 위험한 리스크 아니겠는가!
# stock['Low_Mmin_7'] = stock['Low'].rolling(7, min_periods=1).min().shift(periods=1)

In [0]:
stock.head()

### Shift DataFrame for prediction before 3days

중요한 부분!

순진하게 같은 날짜를 공유하는 Close를 Y취급하는게 아니다!

오늘의 Open High Low Volume이 다 나왔다는 건 당연히 오늘의 Close가 있음. 예측할 필요도 가치도 없다.

**지금까지의 데이터로 n일 뒤의 값을 예측해야 하지 않겠는가!**

계열 예측을 위한 여러방식의 모델링 / 시스템화는 이 강의의 범위를 벗어나므로 다루지 않음.

In [0]:
# n 일 뒤의 데이터를 현재의 row로 끌어와 붙인다.
ndays = 3

stock['Close_3dl'] = stock['Close'].shift(-ndays)
stock.head()

### Drop raws

NaN값으로 찬 로우들을 날리자.

shift기능을 쓰다보면 위 아래로 없는 값들이 나오기 마련.

**주의** 
* 시각화를 위해, 최근 데이터를 날린다.
* 실제 상황에서는 최근데이터 기준 3일 뒤의 값은 예측 대상이다.
* 전처리 했더니 3일 뒤 실제값이 내 데이터에 없다고 날리는 일은 **현실에선 없다**

In [0]:
# any 안에 axis =1 을 통해서 어떤 row 가  NA를 가지고 있는가.
stock.isnull().any(axis=1).head()

In [0]:
stock.head()

In [0]:
stock = stock.dropna()
stock.reset_index()
stock.head()

### Drop columns

* 사용하지 않을 변수들은 제거하자.



In [0]:
fields_to_drop = ['Date', 'Adj Close']
data = stock.drop(fields_to_drop, axis=1)

data.head()

### Scaling target variables

mean shifting & Scaling : 평균을 0으로, 표준편차를 1로<br>
사실 scikit-learn에서 한방에 하는 방법도 있다.<br>
[Missing-Value처리](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Imputer.html), [StandardScaling](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler)<br>
연습삼아 직접 해보자!



In [0]:
quant_features = list(data)
# Store scalings in a dictionary so we can convert back later
scaled_features = {}
for each in quant_features:
    mean, std = data[each].mean(), data[each].std()
    scaled_features[each] = [mean, std]
    data.loc[:, each] = (data[each] - mean)/std

In [0]:
'''
위에서 쓰인 Dictionary 잠깐 짚어보기.
Dictionary의 각각의 원소는 key : value 쌍으로 되어 있다.
'''
scaled_features

In [0]:
data.head()

## Artificial Neural Network

* 아래의 코드는 수정없이 실행하기만 한다.
* 속도가 너무 느릴 경우, 아래의 Linear Regression으로 대체 한다.

### Data Split

In [0]:
# Feature와 Target을 나눈다.

features = data.drop(['Close_3dl'], axis =1)
target = data['Close_3dl']

In [0]:
# Train, Valid, Test로 나눈다.

x_train, x_valid, x_test = features[:-40], features[-40:-20], features[-20:]
y_train, y_valid, y_test = target[:-40], target[-40:-20], target[-20:]

### Scikit-Learn으로 모델링을 하는 4스텝

1. 사용할 method를 불러온다.
2. 사용할 모델을 선언한다.
3. 모델.fit
4. 모델.predict

In [0]:
# 사용할 method 불러오자
from sklearn.neural_network import MLPRegressor

# 사용할 모델 선언하자.

# input node 개수 : feature 개수 (자동)
# hidden layer의 node 개수 : feature 개수로 해뒀음.
# output node 개수 : 1개 (자동)

neural_regression = MLPRegressor(hidden_layer_sizes=(features.shape[1]),
                                activation = 'relu',
                                solver = 'adam',
                                learning_rate_init = 0.0001, #0.00001
                                max_iter = 1000,
                                 random_state=2018, 
                                 
                                 verbose = True, # False로 두면 학습과정을 안볼 수 있다.
                                 early_stopping = True # 모델이 더이상 개선되지 않으면 학습 중단
                                 
                                )

# 모델 학습시키자.
start_time = time.clock()
neural_regression.fit(x_train, y_train )
end_time = time.clock()

print("학습 소요시간 : {:.3f}secs".format(end_time-start_time))

# 예측값은 이렇게 만든다.
neural_regression.predict(x_train)

In [0]:
train_pred, val_pred = neural_regression.predict(x_train), neural_regression.predict(x_valid)

test_pred = neural_regression.predict(x_test)

### 위에서 사이킷런을 사용하여 모델링한 부분을 Keras로 갈아치워라!

**조건**
1. 히든레이어는 2개를 사용하자. 노드 수는 자유롭게.
2. 예측값은 다음의 변수에 담아둬야 한다.
    * train_pred : 트레이닝 셋 위에서의 예측값
    * val_pred : 벨리데이션 셋 위에서의 예측값
    * test_pred : 테스트 셋 위에서의 예측값.
3. 학습에 사용할 데이터는 다음과 같다.
    * x_train,  y_train
4. 벨리데이션 셋, 테스트 셋 데이터는 다음의 변수에 담겨있다.
    * x_valid, y_valid
    * x_test, y_test

In [0]:
########################################################
##############  이곳에 모델링 하시오! ##################
########################################################

























### 주의 : 평가는 현실적으로!

MSE가 낮음에 좋아하지 말것 : 스케일링 했음.

R-squared 값에 현혹되지 말 것.
현실적인 평가를 해야 한다.

이 모델을 주식에 사용한다면, 어떤 평가지표를 사용했어야 했는가. 고민할 것.
1. 예상 수익을 시뮬레이션 해야 함
2. 아무것도 안할 때와 비교해야 함.
3. 내가 투자를 했을때와 비교해야 함.

In [0]:
# Train & Validation 에서의 성능 확인

print("Mean Squared Error on Training set : {0:.5f}".format(mean_squared_error(y_train, train_pred)  ))
print("Mean Squared Error on Validation set : {0:.5f}".format(mean_squared_error(y_valid, val_pred)  ))

print("R-squared Score on Training set : {0:.5f}".format(r2_score(y_train, train_pred)))
print("R-squared Score on Validation set : {0:.5f}".format(r2_score(y_valid, val_pred)))


In [0]:
## 테스트셋의 결과

print("Mean Squared Error on Test set : {0:.5f}".format(mean_squared_error(y_test, test_pred)  ))
print("R-squared Score on Test set : {0:.5f}".format(r2_score(y_test, test_pred)))



In [0]:
## 테스트 셋 위에서의 시각화.


fig, ax = plt.subplots(figsize=(10,5))

# 예측값도 Scaling 되어 있으므로 그것을 원래대로 되돌려주는 과정
mean, std = scaled_features['Close_3dl']

ax.plot((test_pred)*std+mean, label = 'Prediction')
ax.plot( (y_test*std+mean).tolist() , label = 'Real')
ax.set_xlim(right = len(y_test),
           left = 0)
plt.xticks(range(20))
ax.legend()
plt.show()

## 진짜 평가 : 주식 시뮬레이터
* 코드 수정 하지 말 것.
* 모델을 새로 만들었을 때만 실행할 것. 
* 한 모델에 대해서 여러번 반복실행하면 동일 결과가 저장됨

In [0]:
# 트레이더 함수를 만든다.

def trader(m,s,r,p,vol):
    money = m[-1]
    stock = s[-1]
    
    if r > p : # n일 뒤 가격(예측값)이 지금보다 떨어진다면 판다.
        if vol >= stock :
            vol = stock
        
        stock = stock - vol
        money = money + vol*r
    elif r == p : # n일 뒤 가격(예측값)이 지금 값과 같다면 조금 산다.
        vol = vol//4
        stock = stock + vol
        money = money - vol*r
    else : # n일 뒤 가격(예측값)이 지금과 같다면 산다.
        if money < vol*r : # 보유 금액이 거래량 커버가 안된다면
            vol = money // r # 살 수 있을 만큼만 거래량을 줄인다.
        stock = stock + vol
        money = money -vol*r
    return(stock, money)
       

In [0]:
# 초기 자본
my_stock = [100]
my_money = [10000]
# 한번에 사고/파는 거래량
vol = 20

# 시뮬레이션에 사용될 데이터
pred = test_pred*std + mean
real = (y_test*std + mean).tolist()

# 시뮬레이션 사용전 주식 가격
init_value = (y_valid*std + mean).tolist()[-1]

# 시뮬레이션 사용 전 자산 보유량
my_assets = [my_stock[0]*init_value + my_money[0]]

s, m = trader(my_money, my_stock, init_value, pred[ndays-1], vol)
my_stock.append(s)
my_money.append(m)

for i in range(ndays, len(pred)) :
    s, m = trader(my_money, my_stock, real[i-ndays], pred[i], vol )
    my_stock.append(s)
    my_money.append(m)
    
# 마지막 거래 기준, 정보 저장.

asset_change = [my_money[0]+my_stock[0]*init_value]

feature_list = list(data)
feature_list.remove('Close_3dl')

for i in range(1,19):
    asset_change.append(my_money[i]+my_stock[i]*real[i-1])
result_saver = { 'features' : ", ".join(feature_list), 'stock_change' : my_stock, 'money_change' : my_money,
                'asset_change' : asset_change,
                'benefit' : asset_change[-1] - asset_change[0]
               }

result_list.append(result_saver)

In [0]:
print('0 번째 시도는 기준이 되는 모델입니다. 아무런 Feature Engineering이 없습니다.')

for i,ele in enumerate(result_list) :
    print("====== {} 번째 시도 ======".format(i))
    print("사용한 features {}개 : {}".format(len(ele['features']), ele['features']))
    print("시뮬레이션 결과 이득 : {:.3f}$".format(ele['benefit']))
    print("재산 변화 양상")
    for ch in ele['asset_change'] :
        print('{:.1f}'.format(ch), end=' ' )
    print('')
    print('보유한 주식 양상 : {}'.format(ele['stock_change']))
    plt.plot(ele['asset_change'] )
    plt.xticks(range(19))
    plt.show()
    

### 시점 T에서, 당일 포함 7일 이동평균선으로 3일 뒤를 예측하는 경우.

모든 실습이 다 끝나고 구경해보자

In [0]:
MA7 = [115.51714228571434, 115.59999957142863, 115.61714271428578, 115.7100001428572, 115.86428614285718, 115.70142900000005, 115.73857214285718, 115.62285814285717,
       115.42857242857146, 115.29857200000004, 114.92857142857147, 114.47857128571434, 114.49142900000005, 114.45142914285721, 114.03285771428578, 113.73571457142863,
       113.19142914285719, 113.12571500000006, 112.94285800000007, 112.72428671428578]

In [0]:
# 초기 자본
my_stock = [100]
my_money = [10000]
# 한번에 사고/파는 거래량
vol = 20

# 시뮬레이션에 사용될 데이터
pred = MA7
real = (y_test*std + mean).tolist()

# 시뮬레이션 사용전 주식 가격
init_value = (y_valid*std + mean).tolist()[-1]

# 시뮬레이션 사용 전 자산 보유량
my_assets = [my_stock[0]*init_value + my_money[0]]

s, m = trader(my_money, my_stock, init_value, pred[ndays-1], vol)
my_stock.append(s)
my_money.append(m)

for i in range(ndays, len(pred)) :
    s, m = trader(my_money, my_stock, real[i-ndays], pred[i], vol )
    my_stock.append(s)
    my_money.append(m)
    
# 마지막 거래 기준, 정보 저장.

asset_change = [my_money[0]+my_stock[0]*init_value]

for i in range(1,19):
    asset_change.append(my_money[i]+my_stock[i]*real[i-1])
result_saver = { 'features' : "Close", 'stock_change' : my_stock, 'money_change' : my_money,
                'asset_change' : asset_change,
                'benefit' : asset_change[-1] - asset_change[0]
               }

result_list.append(result_saver)

In [0]:
print('0 번째 시도는 기준이 되는 모델입니다. 아무런 Feature Engineering이 없습니다.')
print('마지막 시도가, 7일 이동 평균선을 이용한 3일 뒤 예측입니다.')

for i,ele in enumerate(result_list) :
    print("====== {} 번째 시도 ======".format(i))
    print("사용한 features {}개 : {}".format(len(ele['features']), ele['features']))
    print("시뮬레이션 결과 이득 : {:.3f}$".format(ele['benefit']))
    print("재산 변화 양상")
    for ch in ele['asset_change'] :
        print('{:.1f}'.format(ch), end=' ' )
    print('')
    print('보유한 주식 양상 : {}'.format(ele['stock_change']))
    plt.plot(ele['asset_change'] )
    plt.xticks(range(19))
    plt.show()
    